In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-to-tamil-data/tam.txt


In [3]:
from keras.models import Model
from keras.layers import LSTM,Dense,Input

In [4]:
batch_size=64
samples=10000
latent_dim=256
epochs=100

In [5]:
data_path="../input/english-to-tamil-data/tam.txt"

**Vectorize the data**

In [6]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [11]:
input_characters = sorted(list(input_characters))  #Sorting all characters of English 
target_characters = sorted(list(target_characters)) #Sorting all characters of Tamil
num_encoder_tokens = len(input_characters)          #No of charac's in English
num_decoder_tokens = len(target_characters)         #No of charac's in Tamil
max_encoder_seq_length = max([len(txt) for txt in input_texts])   #Sentence with max words in English
max_decoder_seq_length = max([len(txt) for txt in target_texts])  #Sentence with max words in Tamil

In [12]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 201
Number of unique input tokens: 53
Number of unique output tokens: 54
Max sequence length for inputs: 94
Max sequence length for outputs: 111


In [15]:
print(input_characters)

[' ', '!', "'", ',', '.', '0', '2', '?', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'R', 'S', 'T', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [16]:
print(target_characters)

['\t', '\n', ' ', '!', '(', ')', ',', '.', '0', '2', '?', 'C', 'D', 'அ', 'ஆ', 'இ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', '்']


In [18]:
print(input_texts)

['I slept.', 'Calm down.', "I'll walk.", 'Who is he?', 'Who knows?', 'She smiled.', 'Talk to me!', 'Who is she?', 'Go to sleep.', 'It may rain.', 'She bit him.', 'She hit him.', 'She is kind.', 'She is eight.', 'Where are we?', 'Keep in touch!', 'See you again.', 'Give it to her.', 'I ate too much.', "I'll see to it.", "It's up to you.", 'Leave it to me.', 'Listen to this!', "That's the way.", 'Come and see me.', "Don't lie to me.", 'He began to run.', 'He just arrived.', 'He likes to run.', 'How is your dad?', 'I want to sleep.', "I'm able to run.", 'Raise your hand.', 'What did he say?', 'When can we eat?', 'Come and help us.', 'He is still here.', 'I have to go now.', 'I know that much.', 'I made a mistake.', 'I walk to school.', "That's our house.", 'Those are my CDs.', 'Walk ahead of me.', "We'll follow you.", 'Beware of the dog!', 'He came back soon.', 'He has three sons.', 'I know how to ski.', 'I know what to do.', "I'm kind of happy.", 'Keep to the right.', 'She began to sing.

In [19]:
print(target_texts)

['\tநான் தூங்கினேன்.\n', '\tஅமைதியாக இருங்கள்\n', '\tநான் நடப்பேன்.\n', '\tஅவன் யார்?\n', '\tயாருக்குத் தெரியும்?\n', '\tஅவள் சிரித்தாள்\n', '\tஎன்னிடம் பேசு\n', '\tஅவள் யார்?\n', '\tபோய் தூங்கு\n', '\tமழை பெய்யலாம்\n', '\tஅவள் அவனைக் கடித்தாள்\n', '\tஅவள் அவனைக் அடித்தாள்\n', '\tஅவள் அன்பானவள்\n', '\tஅவளுக்கு எட்டு வயது\n', '\tநாம் எங்கே இருக்கிறோம்?\n', '\tதொடர்பில் இரு\n', '\tமறுபடியும் சந்திப்போம்\n', '\tஅவளிடம் கொடு\n', '\tநான் நிறைய சாப்பிட்டேன்\n', '\tஅதை நான் பார்க்கிறேன்\n', '\tஉன் கையில்தான் இருக்கிறது\n', '\tஎன்னிடம் விட்டுவிடு\n', '\tஇதைக் கேள்\n', '\tஅந்த பக்கம்தான் வழி\n', '\tஎன்னை வந்து பார்\n', '\tஎன்னிடம் பொய் சொல்லாதே\n', '\tஅவன் ஓட ஆரம்பித்தான்\n', '\tஅவன் இப்பொழுதுதான் வந்தான்\n', '\tஅவன் ஓட விருப்பப் படுகிறான்\n', '\tதங்களுடைய தந்தையார் எப்படி இருக்கிறார்கள்?\n', '\tநான் தூங்க விரும்புகிறேன்\n', '\tஎன்னால் ஓட முடிகிறது\n', '\tகையைத் தூக்கு\n', '\tஅவன் என்ன சொன்னான்?\n', '\tஎப்பொழுது நம்மால் சாப்பிட முடியும்\n', '\tவா எங்களுக்கு உதவி செய்\n', '\tஅவன் இன்னும் இருக்கி

In [20]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [21]:
print(input_token_index)

{' ': 0, '!': 1, "'": 2, ',': 3, '.': 4, '0': 5, '2': 6, '?': 7, 'A': 8, 'B': 9, 'C': 10, 'D': 11, 'F': 12, 'G': 13, 'H': 14, 'I': 15, 'J': 16, 'K': 17, 'L': 18, 'M': 19, 'N': 20, 'P': 21, 'R': 22, 'S': 23, 'T': 24, 'W': 25, 'Y': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}


In [22]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '(': 4, ')': 5, ',': 6, '.': 7, '0': 8, '2': 9, '?': 10, 'C': 11, 'D': 12, 'அ': 13, 'ஆ': 14, 'இ': 15, 'உ': 16, 'ஊ': 17, 'எ': 18, 'ஏ': 19, 'ஒ': 20, 'ஓ': 21, 'க': 22, 'ங': 23, 'ச': 24, 'ஜ': 25, 'ஞ': 26, 'ட': 27, 'ண': 28, 'த': 29, 'ந': 30, 'ன': 31, 'ப': 32, 'ம': 33, 'ய': 34, 'ர': 35, 'ற': 36, 'ல': 37, 'ள': 38, 'ழ': 39, 'வ': 40, 'ஷ': 41, 'ஸ': 42, 'ா': 43, 'ி': 44, 'ீ': 45, 'ு': 46, 'ூ': 47, 'ெ': 48, 'ே': 49, 'ை': 50, 'ொ': 51, 'ோ': 52, '்': 53}


**Creating the 3 dimensional array**

(No of input text,Max sequence's length,No of lan's character)

In [23]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [24]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [25]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [27]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
3/3 [==============================] - 3s 871ms/step - loss: 3.2879 - accuracy: 0.4592 - val_loss: 3.0405 - val_accuracy: 0.5889
Epoch 2/100
3/3 [==============================] - 1s 454ms/step - loss: 1.6109 - accuracy: 0.7209 - val_loss: 2.9794 - val_accuracy: 0.5985
Epoch 3/100
3/3 [==============================] - 1s 480ms/step - loss: 1.7518 - accuracy: 0.7610 - val_loss: 1.9920 - val_accuracy: 0.5953
Epoch 4/100
3/3 [==============================] - 1s 480ms/step - loss: 1.0139 - accuracy: 0.7641 - val_loss: 1.7511 - val_accuracy: 0.5950
Epoch 5/100
3/3 [==============================] - 1s 487ms/step - loss: 1.0119 - accuracy: 0.7649 - val_loss: 2.1573 - val_accuracy: 0.5970
Epoch 6/100
3/3 [==============================] - 1s 479ms/step - loss: 0.9820 - accuracy: 0.7671 - val_loss: 1.7170 - val_accuracy: 0.5990
Epoch 7/100
3/3 [==============================] - 1s 471ms/step - loss: 0.9552 - accuracy: 0.7690 - val_loss: 2.0234 - val_accuracy: 0.5968
Epoch 8/100
3

Epoch 59/100
3/3 [==============================] - 1s 491ms/step - loss: 0.6909 - accuracy: 0.8166 - val_loss: 1.2799 - val_accuracy: 0.6579
Epoch 60/100
3/3 [==============================] - 1s 473ms/step - loss: 0.6628 - accuracy: 0.8213 - val_loss: 1.3671 - val_accuracy: 0.6519
Epoch 61/100
3/3 [==============================] - 1s 477ms/step - loss: 0.6715 - accuracy: 0.8188 - val_loss: 1.2842 - val_accuracy: 0.6570
Epoch 62/100
3/3 [==============================] - 1s 479ms/step - loss: 0.8349 - accuracy: 0.8075 - val_loss: 1.4403 - val_accuracy: 0.6412
Epoch 63/100
3/3 [==============================] - 1s 488ms/step - loss: 0.6970 - accuracy: 0.8150 - val_loss: 1.3956 - val_accuracy: 0.6515
Epoch 64/100
3/3 [==============================] - 1s 479ms/step - loss: 0.6561 - accuracy: 0.8244 - val_loss: 1.3476 - val_accuracy: 0.6563
Epoch 65/100
3/3 [==============================] - 1s 486ms/step - loss: 0.6444 - accuracy: 0.8261 - val_loss: 1.2794 - val_accuracy: 0.6581
Epoch 

In [29]:
model.save('tameng.h5')

**Sampling models**

In [31]:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [32]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [33]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [36]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: I slept.
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: Calm down.
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: I'll walk.
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: Who is he?
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: Who knows?
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: She smiled.
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: Talk to me!
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: Who is she?
Decoded sentence: அவன் அவன் வருக்கிறேன்

-
Input sentence: Go to sleep.
Decoded sentence: அவன் அவன் வருக்கிறேன்



KeyboardInterrupt: 